# 0. selenium 설치

In [14]:
!pip install chromedriver_autoinstaller

In [15]:
!pip install selenium

# 1. 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import requests # 크롤링에 사용하는 패키지
from bs4 import BeautifulSoup # html 변환에 사용함


from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

import chromedriver_autoinstaller

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm


# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time


# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 크롤링

##### 참고



- find_elements_by_tag_name:          Copy Element의 tag
- find_elements_by_xpath:             Copy XPath 또는 Copy full XPath
- find_elements_by_id:                Copy Element의 id attribute
- find_elements_by_class_name:        Copy Element의 class attribute
- find_elements_by_link_text:         Copy Element의 text
- find_elements_by_partial_link_text: Copy Element의 text중 일부
- find_elements_by_css_selector:      Copy selector


- '요소' 클릭:        	.click('')   
- '요소' 더블 클릭:	     .double_click('')   
- '텍스트' 입력:	         .send_keys('')  
- '요소'에 '텍스트' 입력:	  .send_keys_to_element('','')  
- '요소'로 마우스 이동:	  .move_to_element('')
- 스크롤 내리기:	         .execute_script("window.scrollto();")
- 실행:                  .perform()

In [31]:
## 크롬 윈도우 설정
# 크롬 드라이버 위치 설정
chrome_path = chromedriver_autoinstaller.install()


# 크롬 옵션
options = webdriver.ChromeOptions()
# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800,1400")

driver = webdriver.Chrome(chrome_path, options=options)

# 실행할 시간 랜덤값 지정
A = np.random.randint(1,3)

## 2-1. 선수 데이터 크롤링


- 선수명
- 급여
- 키
- 몸무게
- 체형
- 개인기
- 주발
- 오버롤 + 전체 스텟
- 시세(일별 금액)
- 클럽경력
- 팀컬러 (팀, 특성, 강화)

*시세의 경우 각 강화 등급별 크롤링! (나머지 값은 후처리로 기입)

In [3]:
# 선수명 csv파일 불러오기
data = pd.read_csv('./data/player.csv')
data

,id,play_name
0,0,데이비드 시먼
1,1,앨런 시어러
2,2,로비 파울러
3,3,로이 킨
4,4,라이언 긱스
...,...,...
59046,59046,A. 크나우프
59047,59047,K. 야키치
59048,59048,M. 구스토
59049,59049,G. 마마르다슈빌리


In [10]:
# 선수 데이터 크롤링
'''
# 크롤링한 결과를 저장할 리스트
name_list = []
salary_list = []
height_list = []
weight_list = []
skill_list = []
prefer_foot = []
overall_list = []
stat_list = []
price_list = []
club_list = []
teamcolor_list = []





for player_name in tqdm(data['play_name'][:5]):
    try:   
        chrome_path = chromedriver_autoinstaller.install()
        driver = webdriver.Chrome(chrome_path, options=options)
        driver.get('https://fifaonline4.nexon.com/datacenter')
        time.sleep(A)
        
        # 선수명 입력
        driver.find_elements_by_css_selector('#searchName').click()
        driver.find_element_by_name('strPlayerName').send_keys(f'{player_name}')
        driver.find_elements_by_css_selector('#form1 > div.search_panel > div.search_input_submit > button.btn_search').click()
        time.sleep(A)
        
    except:
        print('error')
        break
        
'''


  0%|          | 0/5 [00:00<?, ?it/s]

error


## 2-2. 팀 컬러 크롤링



- 팀 컬러 명
- 단계
- 적용 효과

*특성 팀 컬러의 경우'적용 선수' 추가

### 2-2-1. 팀 컬러 목록 part

In [51]:
## 팀 컬러 목록 크롤링
# 팀 컬러 타입 리스트
teamcolor_club = []
teamcolor_nation = []
teamcolor_reinforce = []
teamcolor_relation = []
teamcolor_special =[]

# 팀 컬러 정보 목록 크롤링 파트 함수
def TC_list_crawling():
    driver.find_element_by_class_name('btn_search').click()
    time.sleep(A)

    locates = 'div.name'
    elements = driver.find_elements_by_css_selector(locates)
    
    return elements



# 홈페이지 열기
url = 'https://fifaonline4.nexon.com/datacenter/teamcolor?strTeamColorCategory=&strTeamColorType=&strTeamColorName='
driver.get(url)
time.sleep(A)


# # 팝업창 닫기 (팝업창 오류 시 사용)
# driver.find_element_by_xpath('//*[@id="wrapper"]/div[1]/a/span').click()
# time.sleep(A)

# 팀 컬러 정보 목록 저장
for x in tqdm(range(2,7)):
        driver.find_element_by_xpath(f'//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[{x}]/label').click()
        time.sleep(A)
        
        elements = TC_list_crawling()

        ## 클럽
        if x == 2:
                for element in elements:
                        teamcolor_club.append(element.text)
                
        elif x == 3:
                for element in elements:
                        teamcolor_nation.append(element.text)
        
        elif x == 4:
                for element in elements:
                        teamcolor_reinforce.append(element.text)
                        
        elif x == 5:
                for element in elements:
                        teamcolor_relation.append(element.text)
                        
        elif x == 6:
                for element in elements:
                        teamcolor_special.append(element.text)
        
        else:
                print('not in range')
        
        driver.find_element_by_class_name('btn_reset').click()
        time.sleep(A)
        

  0%|          | 0/5 [00:00<?, ?it/s]

In [18]:
# 팀 컬러 목록 데이터 프레임 저장
TC_club = pd.DataFrame(data = teamcolor_club, columns = ['team color'])
TC_nation = pd.DataFrame(data = teamcolor_nation, columns = ['team color'])
TC_reinforce = pd.DataFrame(data = teamcolor_reinforce, columns = ['team color'])
TC_relation = pd.DataFrame(data = teamcolor_relation, columns = ['team color'])
TC_special = pd.DataFrame(data = teamcolor_special, columns = ['team color'])

# # csv로 파일 저장
# TC_club.to_csv("TC_club.csv", encoding='utf-8-sig', index = False)
# TC_nation.to_csv("TC_nation.csv", encoding='utf-8-sig', index = False)
# TC_reinforce.to_csv("TC_reinforce.csv", encoding='utf-8-sig', index = False)
# TC_relation.to_csv("TC_relation.csv", encoding='utf-8-sig', index = False)
# TC_special.to_csv("TC_special.csv", encoding='utf-8-sig', index = False)


### 2-2-2. 팀 컬러 세부 사항 part

In [112]:
# csv 파일 불러오기
TC_club = pd.read_csv('./data/team_color_list/TC_club.csv')
TC_nation = pd.read_csv('./data/team_color_list/TC_nation.csv')
TC_reinforce = pd.read_csv('./data/team_color_list/TC_reinforce.csv')
TC_relation = pd.read_csv('./data/team_color_list/TC_relation.csv')
TC_special = pd.read_csv('./data/team_color_listTC_special.csv')

In [36]:
# 팀 컬러 타입 모음
teamcolor_type = [teamcolor_club,teamcolor_nation,teamcolor_relation,teamcolor_special,teamcolor_reinforce]
#teamcolor_type = [TC_club,TC_nation,TC_relation,TC_special,TC_reinforce]


In [54]:
# css selector 위치 입력기
def location_inputor(num,type):
    if type == 'div':
        temp_1 = driver.find_element_by_css_selector('div > div.lv_content > div.desc').text
        temp_2 = driver.find_element_by_css_selector('div > div.lv_content > div.ap_list').text
        temp_2 = temp_2.replace('\n',' / ').replace('-','')
        
    elif type == 'div.level.lvu':
        temp_1 = driver.find_element_by_css_selector(f'div.level.lvu{num} > div.lv_content > div.desc').text
        temp_2 = driver.find_element_by_css_selector(f'div.level.lvu{num} > div.lv_content > div.ap_list').text
        temp_2 = temp_2.replace('\n',' / ').replace('-','')
        
    elif type == 'div.level.lv':    
        temp_1 = driver.find_element_by_css_selector(f'div.level.lv{num} > div.lv_content > div.desc').text
        temp_2 = driver.find_element_by_css_selector(f'div.level.lv{num} > div.lv_content > div.ap_list').text
        temp_2 = temp_2.replace('\n',' / ').replace('-','')
        
    return temp_1, temp_2

In [55]:
## 팀 컬러 타입 별 적용 조건 & 효과 저장
# 홈페이지 열기
url = 'https://fifaonline4.nexon.com/datacenter/teamcolor?strTeamColorCategory=&strTeamColorType=&strTeamColorName='
driver.get(url)

count = 0

for teamcolors in tqdm(teamcolor_type):
   # 크롤링한 결과를 저장할 리스트
   teamcolor_info = []
   step_1_require_player_nums = []
   step_2_require_player_nums = []
   step_3_require_player_nums = []
   step_4_require_player_nums = []
   step_1_effects = []
   step_2_effects = []
   step_3_effects = []
   step_4_effects = []
   
   for x in tqdm(teamcolors):
      # # 팝업창 닫기
      # driver.find_element_by_xpath('//*[@id="wrapper"]/div[1]/a/span').click()
      # time.sleep(A)
      try:
         
         # 팀 컬러 타입 선택
         if teamcolors == teamcolor_club:
            driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[2]/label').click()
         elif teamcolors == teamcolor_nation:
            driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[3]/label').click() 
         elif teamcolors == teamcolor_reinforce:
            driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[4]/label').click()
         elif teamcolors == teamcolor_relation:
            driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[5]/label').click() 
         elif teamcolors == teamcolor_special:
            driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[6]/label').click()  
         
         time.sleep(1)
         
         
         # 팀 컬러 입력 & 검색
         driver.find_element_by_class_name('ui-autocomplete-input').click()
         driver.find_element_by_class_name('ui-autocomplete-input').send_keys(x)
         driver.find_element_by_class_name('btn_search').click()
         time.sleep(A)
         
         
         # 단계 추출 (for 문 돌릴 갯수 정하기 용도)
         number = driver.find_element_by_css_selector('div.teamcolor_item_list > div > div.level').text
         number = int(number.replace('단계',''))


         # 세부 설명 들어가기
         driver.find_element_by_class_name('name').click()
         driver.find_element_by_class_name('btn_detail_link').click()
         time.sleep(A) 


         # 팀 컬러 설명 저장
         info = driver.find_element_by_css_selector('#teamcolorPop > div > div.header > div > span').text
         teamcolor_info.append(info)
         time.sleep(1)
         
         
         
         # 단계 별 적용 조건 & 적용 효과 저장
         for num in range(1,5):
            # 1단계
            if num == 1:
               if number == 1:
                  # 'num'과 아래의 타입 중 하나를 골라서 넣으세요!
                  # type = 'div', 'div.level.lvu', 'div.level.lv'
                  temp_1, temp_2 = location_inputor(num,'div')
                  
                  step_1_require_player_nums.append(temp_1)
                  step_1_effects.append(temp_2)
                  
               elif number > 1:   
                  if teamcolors == teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lvu')
                  
                  elif teamcolors != teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lv')
               
                  step_1_require_player_nums.append(temp_1)
                  step_1_effects.append(temp_2)
               else:
                  step_1_require_player_nums.append('-')
                  step_1_effects.append('-') 

            # 2단계
            elif num == 2:
               if num <= number:
                  if teamcolors == teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lvu')
                  
                  elif teamcolors != teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lv')
                  
                  step_2_require_player_nums.append(temp_1)
                  step_2_effects.append(temp_2)
                  
               else:
                  step_2_require_player_nums.append('-')
                  step_2_effects.append('-') 
               
            # 3단계
            elif num == 3:
               if num <= number:
                  if teamcolors == teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lvu')
                  
                  elif teamcolors != teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lv')
                  
                  step_3_require_player_nums.append(temp_1)
                  step_3_effects.append(temp_2)
                  
               else:
                  step_3_require_player_nums.append('-')
                  step_3_effects.append('-') 

            # 4단계
            elif num == 4:
               if num <= number:
                  if teamcolors == teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lvu')
                  
                  elif teamcolors != teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lv')
                  
                  step_4_require_player_nums.append(temp_1)
                  step_4_effects.append(temp_2)
                  
               else:
                  step_4_require_player_nums.append('-')
                  step_4_effects.append('-') 

            else:
               print('error')
               
            time.sleep(1)
            
         # 세부 설명 나오기
         driver.find_element_by_class_name('btn_close').click()
         time.sleep(1)

         # 검색어 초기화 
         driver.find_element_by_class_name('btn_reset').click()
         time.sleep(1)
      
      except:
         print('error: 팀 컬러 선택 & 상세 정보 클릭')
         break
      
   
   
   ## 팀 컬러 하나 완료하면 csv로 저장            
   # dict로 변형 & 데이터 프레임화
   dict_temp = {  '팀 컬러': [val for val in teamcolors],
                  '팀 컬러 설명': [val for val in teamcolor_info],             
                  '1단계 적용 조건': [val for val in step_1_require_player_nums],
                  '2단계 적용 조건': [val for val in step_2_require_player_nums],
                  '3단계 적용 조건': [val for val in step_3_require_player_nums],
                  '4단계 적용 조건': [val for val in step_4_require_player_nums],
                  '1단계 효과': [val for val in step_1_effects], 
                  '2단계 효과': [val for val in step_2_effects],
                  '3단계 효과': [val for val in step_3_effects],
                  '4단계 효과': [val for val in step_4_effects]
   }
   columns = ['팀 컬러','팀 컬러 설명','1단계 적용 조건','2단계 적용 조건','3단계 적용 조건','4단계 적용 조건','1단계 효과','2단계 효과','3단계 효과','4단계 효과']
   
   teamcolor_naming = ['클럽_팀_컬러','국가_팀_컬러','강화_팀_컬러','관계_팀_컬러','스페셜_팀_컬러']

   globals()[f'df_{teamcolor_naming[count]}'] = pd.DataFrame(data = dict_temp, columns=columns)
   count += 1
   
   time.sleep(A)
   


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/206 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [59]:
## 데이터 확인  
# df_클럽_팀_컬러
# df_국가_팀_컬러
# df_강화_팀_컬러
# df_관계_팀_컬러
# df_스페셜_팀_컬러

,팀 컬러,팀 컬러 설명,1단계 적용 조건,2단계 적용 조건,3단계 적용 조건,4단계 적용 조건,1단계 효과,2단계 효과,3단계 효과,4단계 효과
0,19 UEFA Champions League,19 UEFA Champions League 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
1,20 K League Best,20 K League Best 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
2,20 UEFA Champions League,20 UEFA Champions League 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
3,21 K League Best,21 K League Best 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
4,21 UEFA Champions League,21 UEFA Champions League 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
5,Ones To Watch,Ones To Watch 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
6,TOTS,역대 TOTS에 선정된 선수들로 구성된 팀컬러입니다,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / 민첩성 +2 / 스태미너 +1 /,-
7,TOTY,역대 TOTY에 선정된 선수들로 구성된 팀컬러입니다,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / 스태미너 +2 / 적극성 +1 /,-


In [60]:
## csv파일로 저장
# df_클럽_팀_컬러.to_csv('클럽팀컬러.csv', encoding='utf-8-sig', index = False)
# df_국가_팀_컬러.to_csv('국가팀컬러.csv', encoding='utf-8-sig', index = False)
# df_강화_팀_컬러.to_csv('강화팀컬러.csv', encoding='utf-8-sig', index = False)
# df_관계_팀_컬러.to_csv('관계팀컬러.csv', encoding='utf-8-sig', index = False)
# df_스페셜_팀_컬러.to_csv('스페셜팀컬러.csv', encoding='utf-8-sig', index = False)


In [ ]:
# 관계 팀 컬러 적용 선수 목록 추가
season_name = []
player_name = []
        
# '관계 팀 컬러'일 때, 선수 목록도 저장
if  teamcolors in teamcolor_relation:
   temp_0 = driver.find_element_by_class_name('info_top').text()
   require_season.append(temp_0)
else:
   pass